## News classification using Gensim word vectors

In [1]:
import gensim.downloader as  api

In [2]:
wv = api.load('word2vec-google-news-300')

In [3]:
wv.similarity(w1="man", w2="dog")
w1 =wv["good"]
w1

array([ 0.04052734,  0.0625    , -0.01745605,  0.07861328,  0.03271484,
       -0.01263428,  0.00964355,  0.12353516, -0.02148438,  0.15234375,
       -0.05834961, -0.10644531,  0.02124023,  0.13574219, -0.13183594,
        0.17675781,  0.27148438,  0.13769531, -0.17382812, -0.14160156,
       -0.03076172,  0.19628906, -0.03295898,  0.125     ,  0.25390625,
        0.12695312, -0.15234375,  0.03198242,  0.01135254, -0.01361084,
       -0.12890625,  0.01019287,  0.23925781, -0.08447266,  0.140625  ,
        0.13085938, -0.04516602,  0.06494141,  0.02539062,  0.05615234,
        0.24609375, -0.20507812,  0.23632812, -0.00860596, -0.02294922,
        0.05078125,  0.10644531, -0.03564453,  0.08740234, -0.05712891,
        0.08496094,  0.23535156, -0.10107422, -0.03564453, -0.04736328,
        0.04736328, -0.14550781, -0.10986328,  0.14746094, -0.23242188,
       -0.07275391,  0.19628906, -0.37890625, -0.07226562,  0.04833984,
        0.11914062,  0.06103516, -0.12109375, -0.27929688,  0.05

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv('D:/ML Coding Ninza/Coding/Dataset/news.csv')

In [7]:
print(df.shape)
df.head()

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
print(df.text[4])
df.label.value_counts()

It's primary day in New York and front-runners Hillary Clinton and Donald Trump are leading in the polls.

Trump is now vowing to win enough delegates to clinch the Republican nomination and prevent a contested convention. But Sens.Ted Cruz, R-Texas, Bernie Sanders, D-Vt., and Ohio Gov. John Kasich and aren't giving up just yet.

A big win in New York could tip the scales for both the Republican and Democratic front-runners in this year's race for the White House. Clinton and Trump have each suffered losses in recent contests, shifting the momentum to their rivals.

"We have won eight out of the last nine caucuses and primaries! Cheer!" Sanders recently told supporters.

While wins in New York for Trump and Clinton are expected, the margins of those victories are also important.

Trump needs to capture more than 50 percent of the vote statewide if he wants to be positioned to win all of the state's 95 GOP delegates. That would put him one step closer to avoiding a contested convention.

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [9]:
df.drop(df.columns[0:2], inplace=True, axis=1)
df.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [10]:
df['target'] = df['label'].map({
    'FAKE': 0, 
    'REAL': 1
})
df.head()

,text,label,target
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0
2,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0
4,It's primary day in New York and front-runners...,REAL,1


In [12]:
wv.most_similar('cat')

[('cats', 0.8099379539489746),
 ('dog', 0.760945737361908),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326234579086304),
 ('beagle', 0.7150582671165466),
 ('puppy', 0.7075453400611877),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931973457336),
 ('chihuahua', 0.6709762215614319)]

### preprocessing

In [13]:
import spacy 
nlp = spacy.load('en_core_web_lg')

In [62]:
def preprocess_and_vectorize(text):
    filtered_token = []
    doc = nlp(text)
    for token in doc:
        if not token.is_stop and not token.is_punct:
            filtered_token.append(token.lemma_)
    
    # for word in filtered_token:
        # vector.append(wv['word'])
    
    
    return wv.get_mean_vector(filtered_token)


In [63]:
preprocess_and_vectorize("i am going for a walked. Do you care to join me").shape


(300,)

In [64]:
wv.get_mean_vector( ['cat' ,'dog'],pre_normalize=False)[:3]

array([ 0.03179932,  0.09088135, -0.2290039 ], dtype=float32)

In [65]:
import numpy as np

In [66]:
w1 = wv['cat']
w2 = wv['dog']
np.mean([w1, w2], axis=0)[:3]


array([ 0.03179932,  0.09088135, -0.2290039 ], dtype=float32)

In [67]:
df['vector'] = df['text'].apply(lambda text: preprocess_and_vectorize(text))

In [74]:
df.vector[4]

array([ 1.70995723e-02,  1.23965414e-02, -3.52646969e-03, -2.34477688e-03,
       -5.17879799e-03, -2.99869385e-02, -7.15782866e-03, -1.35092493e-02,
        2.50880402e-02,  2.63370518e-02, -1.84035953e-02, -1.56931346e-03,
       -6.60459371e-03,  3.62214521e-02, -2.09851358e-02,  2.63604652e-02,
        2.36558747e-02,  4.40453291e-02,  1.61714014e-02, -2.69992296e-02,
        1.33547082e-03,  1.22623686e-02,  2.66453475e-02, -8.09775945e-03,
        2.63188370e-02, -8.64748005e-03, -3.34688798e-02, -9.28568840e-03,
        1.86541900e-02, -8.10821820e-03,  4.41827206e-03,  6.24958239e-03,
       -2.36203466e-02, -1.92681141e-02,  2.71963552e-02,  1.78588333e-03,
        1.81373321e-02,  1.33230481e-02, -3.89346271e-03,  3.32208499e-02,
        3.33763920e-02, -3.36737819e-02,  4.47561853e-02,  2.06846725e-02,
       -4.42348048e-02, -9.57671180e-03, -4.62444127e-03,  7.82934111e-03,
        2.52343202e-03,  2.95096301e-02, -9.99508891e-03,  2.58867387e-02,
       -6.96214847e-03, -

In [75]:
df.head()

,text,label,target,vector
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0,"[0.024779696, 0.012828968, -0.00094372977, 0.0..."
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0,"[0.027095925, 0.010018359, -0.003129262, 0.026..."
2,U.S. Secretary of State John F. Kerry said Mon...,REAL,1,"[0.007654696, 0.017431496, 0.01465004, 0.02029..."
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0,"[0.026231475, 0.0026343616, -0.004902629, 0.01..."
4,It's primary day in New York and front-runners...,REAL,1,"[0.017099572, 0.012396541, -0.0035264697, -0.0..."


### Training the data

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.vector, df.target, 
test_size=0.2, random_state=2003, stratify=df.target)

In [77]:
X_train.shape, X_test.shape, y_train.shape, y_train.shape

((5068,), (1267,), (5068,), (5068,))

In [79]:
X_train.values

array([array([ 1.64968222e-02,  1.54385790e-02,  5.54654421e-03,  1.37493259e-03,
              -2.77393702e-02, -2.39225551e-02,  9.92960483e-03, -2.98539530e-02,
               4.22187559e-02, -1.08222105e-02, -3.05113886e-02, -2.16162694e-03,
              -2.39147060e-03,  2.61639655e-02, -5.29644042e-02,  2.85068080e-02,
              -7.93061554e-05,  2.29024608e-02, -7.49576837e-03, -8.18362087e-02,
               1.85181890e-02,  3.71239409e-02, -1.37076678e-03, -2.78396960e-02,
              -3.68070602e-02, -3.85002494e-02, -2.05816124e-02,  1.73196215e-02,
               2.23900378e-02,  3.17006782e-02,  7.72407511e-03,  2.30512396e-02,
              -3.62387411e-02,  5.61018055e-03,  2.42047440e-02, -2.47776061e-02,
               1.06984302e-02,  3.52591164e-02,  1.28122082e-03,  2.03686059e-02,
               2.16993596e-02, -4.24958952e-02,  2.16555912e-02,  1.84704419e-02,
              -4.43166941e-02, -2.66845077e-02, -1.81479994e-02,  1.00787086e-02,
              -4

In [82]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()

In [86]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)
X_train_2d

array([[ 0.01649682,  0.01543858,  0.00554654, ..., -0.03671877,
        -0.05454308,  0.02157441],
       [ 0.01174226,  0.00730912, -0.00741954, ...,  0.00219593,
         0.00153885, -0.0105866 ],
       [ 0.02343049,  0.01363296, -0.00836552, ..., -0.01096017,
         0.01038288,  0.00658286],
       ...,
       [ 0.00781972,  0.01628617,  0.00333883, ..., -0.00257808,
         0.01264465, -0.00119215],
       [ 0.01630071,  0.0028702 ,  0.00181022, ..., -0.01200747,
         0.01151585,  0.00979295],
       [ 0.01683513,  0.01041802,  0.00702362, ..., -0.00997206,
         0.0114024 ,  0.003102  ]], dtype=float32)

In [87]:
X_train_scaled = scaler.fit_transform(X_train_2d)
X_test_scaled = scaler.fit_transform(X_test_2d)
X_train_scaled

array([[0.47418422, 0.27450725, 0.70521164, ..., 0.47171444, 0.        ,
        0.7424964 ],
       [0.42644846, 0.23356505, 0.595996  , ..., 0.732085  , 0.49702486,
        0.51876116],
       [0.5437981 , 0.26541364, 0.5880279 , ..., 0.64406013, 0.5754049 ,
        0.63820434],
       ...,
       [0.38706627, 0.27877596, 0.6866157 , ..., 0.700143  , 0.5954498 ,
        0.58411574],
       [0.47221524, 0.21120946, 0.67373997, ..., 0.6370528 , 0.5854458 ,
        0.660536  ],
       [0.47758085, 0.24922237, 0.71765333, ..., 0.65067136, 0.5844404 ,
        0.61398894]], dtype=float32)

In [92]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [96]:
clf = GradientBoostingClassifier()
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

In [97]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [98]:
print(classification_report(y_test, y_pred))
accuracy_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.51      1.00      0.68       633
           1       1.00      0.05      0.09       634

    accuracy                           0.52      1267
   macro avg       0.76      0.52      0.38      1267
weighted avg       0.76      0.52      0.38      1267



0.5232833464877664